In [1]:
import os
os.environ["CHAINER_TYPE_CHECK"] = "0"

import math
import random
import numpy as np
import chainer
from chainer import cuda, Chain, optimizers, Variable, serializers, Link
import chainer.functions as F
import chainer.links as L
import cupy
import cv2

import time

#from cvbatch2 import CVBatch2 

In [2]:
xp = cuda.cupy
#print(chainer.functions.Linear(1,1).type_check_enable)

In [3]:
class EqualizedLinear(chainer.Chain):
    def __init__(self, in_ch, out_ch):
        w = chainer.initializers.Normal(1.0) # equalized learning rate
        self.inv_c = np.sqrt(2.0/in_ch)
        super(EqualizedLinear, self).__init__()
        with self.init_scope():
            self.c = L.Linear(in_ch, out_ch, initialW=w)
            
    def __call__(self, x):
        return self.c(self.inv_c * x)

In [4]:
class EqualizedConv2d(chainer.Chain):
    def __init__(self, in_ch, out_ch, ksize, stride, pad):
        w = chainer.initializers.Normal(1.0) # equalized learning rate
        self.inv_c = np.sqrt(2.0/(in_ch*ksize**2))
        super(EqualizedConv2d, self).__init__()
        with self.init_scope():
            self.c = L.Convolution2D(in_ch, out_ch, ksize, stride, pad, initialW=w)
            
    def __call__(self, x):
        return self.c(self.inv_c * x)

In [5]:
class GeneratorBlock(chainer.Chain):
    def __init__(self, in_ch, out_ch):
        super(GeneratorBlock, self).__init__()
        with self.init_scope():
            self.c0 = EqualizedConv2d(in_ch, out_ch, 3, 1, 1)
            self.c1 = EqualizedConv2d(out_ch, out_ch, 3, 1, 1)
            
    def __call__(self, x):
        h = F.unpooling_2d(x, 2, 2, 0, outsize=(x.shape[2]*2, x.shape[3]*2))
        h = F.leaky_relu(feature_vector_normalization(self.c0(h)))
        h = F.leaky_relu(feature_vector_normalization(self.c1(h)))
        return h

In [6]:
def feature_vector_normalization(x, eps=1e-8):
    # x: (B, C, H, W)
    alpha = 1.0 / F.sqrt(F.mean(x*x, axis=1, keepdims=True) + eps)
    return F.broadcast_to(alpha, x.data.shape) * x


def minibatch_std(x):
    m = F.mean(x, axis=0, keepdims=True)
    v = F.mean((x - F.broadcast_to(m, x.shape))*(x - F.broadcast_to(m, x.shape)), axis=0, keepdims=True)
    std = F.mean(F.sqrt(v + 1e-8), keepdims=True)
    std = F.broadcast_to(std, (x.shape[0], 1, x.shape[2], x.shape[3]))
    return F.concat([x, std], axis=1)

In [7]:
class Generator(chainer.Chain):
    def __init__(self, n_hidden=512, max_stage=12):
        super(Generator, self).__init__()
        self.n_hidden = n_hidden
        self.R = (512,512,512,512,256,128,64,32)
        self.max_stage = max_stage
        with self.init_scope():
            
            self.c0 = EqualizedConv2d(n_hidden, self.R[0], 4, 1, 3)
            self.c1 = EqualizedConv2d(self.R[0], self.R[0], 3, 1, 1)
            self.out0 = EqualizedConv2d(self.R[0], 3, 1, 1, 0)

            self.b1 = GeneratorBlock(self.R[0], self.R[1])
            self.out1 = EqualizedConv2d(self.R[1], 3, 1, 1, 0)
            self.b2 = GeneratorBlock(self.R[1], self.R[2])
            self.out2 = EqualizedConv2d(self.R[2], 3, 1, 1, 0)
            self.b3 = GeneratorBlock(self.R[2], self.R[3])
            self.out3 = EqualizedConv2d(self.R[3], 3, 1, 1, 0)
            self.b4 = GeneratorBlock(self.R[3], self.R[4])
            self.out4 = EqualizedConv2d(self.R[4], 3, 1, 1, 0)
            self.b5 = GeneratorBlock(self.R[4], self.R[5])
            self.out5 = EqualizedConv2d(self.R[5], 3, 1, 1, 0)
            self.b6 = GeneratorBlock(self.R[5], self.R[6])
            self.out6 = EqualizedConv2d(self.R[6], 3, 1, 1, 0)
            self.b7 = GeneratorBlock(self.R[6], self.R[7])
            self.out7 = EqualizedConv2d(self.R[7], 3, 1, 1, 0)

    def make_hidden(self, batchsize):
        xp = self.xp
        z = xp.random.normal(size=(batchsize, self.n_hidden, 1, 1)) \
            .astype(np.float32)
        z /= xp.sqrt(xp.sum(z*z, axis=1, keepdims=True)/self.n_hidden + 1e-8)
        return z

    def __call__(self, z, stage):
        # stage0: c0->c1->out0
        # stage1: c0->c1-> (1-a)*(up->out0) + (a)*(b1->out1)
        # stage2: c0->c1->b1->out1
        # stage3: c0->c1->b1-> (1-a)*(up->out1) + (a)*(b2->out2)
        # stage4: c0->c1->b2->out2
        # ...

        stage = min(stage, self.max_stage)
        alpha = stage - math.floor(stage)
        stage = math.floor(stage)

        h = F.reshape(z,(len(z), self.n_hidden, 1, 1))
        h = F.leaky_relu(feature_vector_normalization(self.c0(h)))
        h = F.leaky_relu(feature_vector_normalization(self.c1(h)))

        for i in range(1, int(stage//2+1)):
            h = getattr(self, "b%d"%i)(h)

        if int(stage)%2==0:
            out = getattr(self, "out%d"%(stage//2))
            x = out(h)
        else:
            out_prev = getattr(self, "out%d"%(stage//2))
            out_curr = getattr(self, "out%d"%(stage//2+1))
            b_curr = getattr(self, "b%d"%(stage//2+1))

            x_0 = out_prev(F.unpooling_2d(h, 2, 2, 0, outsize=(2*h.shape[2], 2*h.shape[3])))
            x_1 = out_curr(b_curr(h))
            x = (1.0-alpha)*x_0 + alpha*x_1

        if chainer.configuration.config.train:
            return x
        else:
            scale = int(256 // x.data.shape[2])
            return F.unpooling_2d(x, scale, scale, 0, outsize=(256,256))

In [8]:
class DiscriminatorBlock(chainer.Chain):
    # conv-conv-downsample
    def __init__(self, in_ch, out_ch, pooling_comp):
        super(DiscriminatorBlock, self).__init__()
        self.pooling_comp = pooling_comp
        with self.init_scope():
            self.c0 = EqualizedConv2d(in_ch, in_ch, 3, 1, 1)
            self.c1 = EqualizedConv2d(in_ch, out_ch, 3, 1, 1)
            
    def __call__(self, x):
        h = F.leaky_relu((self.c0(x)))
        h = F.leaky_relu((self.c1(h)))
        h = self.pooling_comp * F.average_pooling_2d(h, 2, 2, 0)
        return h

In [9]:
class Discriminator(chainer.Chain):
    def __init__(self, max_stage=12, pooling_comp=1.0):
        super(Discriminator, self).__init__()
        self.max_stage = max_stage
        self.pooling_comp = pooling_comp # compensation of ave_pool is 0.5-Lipshitz
        
        self.R = (32,64,128,256,512,512,512,512)
        
        with self.init_scope():
            self.in7 = EqualizedConv2d(3, self.R[0], 1, 1, 0)
            self.b7 = DiscriminatorBlock(self.R[0], self.R[1], pooling_comp)
            self.in6 = EqualizedConv2d(3, self.R[1], 1, 1, 0)
            self.b6 = DiscriminatorBlock(self.R[1], self.R[2], pooling_comp)
            self.in5 = EqualizedConv2d(3, self.R[2], 1, 1, 0)
            self.b5 = DiscriminatorBlock(self.R[2], self.R[3], pooling_comp)
            self.in4 = EqualizedConv2d(3, self.R[3], 1, 1, 0)
            self.b4 = DiscriminatorBlock(self.R[3], self.R[4], pooling_comp)
            self.in3 = EqualizedConv2d(3, self.R[4], 1, 1, 0)
            self.b3 = DiscriminatorBlock(self.R[4], self.R[5], pooling_comp)
            self.in2 = EqualizedConv2d(3, self.R[5], 1, 1, 0)
            self.b2 = DiscriminatorBlock(self.R[5], self.R[6], pooling_comp)
            self.in1 = EqualizedConv2d(3, self.R[6], 1, 1, 0)
            self.b1 = DiscriminatorBlock(self.R[6], self.R[7], pooling_comp)
            self.in0 = EqualizedConv2d(3, self.R[7], 1, 1, 0)

            self.out0 = EqualizedConv2d(self.R[7]+1, self.R[7], 3, 1, 1)
            self.out1 = EqualizedConv2d(self.R[7], self.R[7], 4, 1, 0)
            self.out2 = EqualizedLinear(self.R[7], 1)

    def __call__(self, x, stage):
        # stage0: in0->m_std->out0_0->out0_1->out0_2
        # stage1: (1-a)*(down->in0) + (a)*(in1->b1) ->m_std->out0->out1->out2
        # stage2: in1->b1->m_std->out0_0->out0_1->out0_2
        # stage3: (1-a)*(down->in1) + (a)*(in2->b2) ->b1->m_std->out0->out1->out2
        # stage4: in2->b2->b1->m_std->out0->out1->out2
        # ...

        stage = min(stage, self.max_stage)
        alpha = stage - math.floor(stage)
        stage = math.floor(stage)

        if int(stage)%2==0:
            fromRGB = getattr(self, "in%d"%(stage//2))
            h = F.leaky_relu(fromRGB(x))
        else:
            fromRGB0 = getattr(self, "in%d"%(stage//2))
            fromRGB1 = getattr(self, "in%d"%(stage//2+1))
            b1 = getattr(self, "b%d"%(stage//2+1))


            h0 = F.leaky_relu(fromRGB0(self.pooling_comp * F.average_pooling_2d(x, 2, 2, 0)))
            h1 = b1(F.leaky_relu(fromRGB1(x)))
            h = (1-alpha)*h0 + alpha*h1

        for i in range(int(stage // 2), 0, -1):
            h = getattr(self, "b%d" % i)(h)

        h = minibatch_std(h)
        h = F.leaky_relu((self.out0(h)))
        h = F.leaky_relu((self.out1(h)))
        return self.out2(h)

In [10]:
def copy_param(target_link, source_link):
    """Copy parameters of a link to another link."""
    target_params = dict(target_link.namedparams())
    for param_name, param in source_link.namedparams():
        target_params[param_name].data[:] = param.data

    # Copy Batch Normalization's statistics
    target_links = dict(target_link.namedlinks())
    for link_name, link in source_link.namedlinks():
        if isinstance(link, L.BatchNormalization):
            target_bn = target_links[link_name]
            target_bn.avg_mean[:] = link.avg_mean
            target_bn.avg_var[:] = link.avg_var


def soft_copy_param(target_link, source_link, tau):
    """Soft-copy parameters of a link to another link."""
    target_params = dict(target_link.namedparams())
    for param_name, param in source_link.namedparams():
        target_params[param_name].data[:] *= (1 - tau)
        target_params[param_name].data[:] += tau * param.data

    # Soft-copy Batch Normalization's statistics
    target_links = dict(target_link.namedlinks())
    for link_name, link in source_link.namedlinks():
        if isinstance(link, L.BatchNormalization):
            target_bn = target_links[link_name]
            target_bn.avg_mean[:] *= (1 - tau)
            target_bn.avg_mean[:] += tau * link.avg_mean
            target_bn.avg_var[:] *= (1 - tau)
            target_bn.avg_var[:] += tau * link.avg_var


In [11]:
init=155000

gen = Generator(512)
generator_smooth = Generator(512)
if init != 0:
    serializers.load_npz("generator.npz", gen)
    

gen.compute_accuracy = False
generator_smooth.compute_accuracy = False

gpu = 0
chainer.cuda.get_device(gpu).use()

gen.to_gpu(gpu) # GPUを使うための処理
generator_smooth.to_gpu(gpu) # GPUを使うための処理

# コピー
copy_param(generator_smooth, gen)

if init != 0:
    serializers.load_npz("generator_smooth.npz", generator_smooth)

gen_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99)
#gen_optimizer = optimizers.RMSprop(lr=0.00005)
gen_optimizer.setup(gen)



# Critic
cri = Discriminator()
cri.compute_accuracy = False
if init != 0:
    serializers.load_npz("critic.npz", cri)
cri.to_gpu(gpu) # GPUを使うための処理

cri_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99) # 0.001
#cri_optimizer = optimizers.RMSprop(lr=0.00005)
cri_optimizer.setup(cri)
#cri_optimizer.add_hook(chainer.optimizer.WeightDecay(0.0001)) # 0.005




In [12]:
#cv_batch2 = CVBatch2()
#cv_batch2.loadImages('train/', (256, 256))

#total_num = cv_batch2.num

INPUT_W = 256
INPUT_H = 256
INPUT_C = 3
INPUT_NUM = INPUT_W*INPUT_H*INPUT_C
#TRAIN_NUM = 288*2

# 読み込み、前処理
def img_flat_read(fpass):
    img = cv2.imread(fpass)

    # INPUT_WxINPUT_Hにリサイズ
    img_64 = cv2.resize(img, (INPUT_W, INPUT_H), interpolation = cv2.INTER_AREA)
    
    # 1列にし-1～1のfloatに
    img_flat = 2.0 * img_64 / 255.0 - 1.0
    img_flat_rev = 2.0 * cv2.flip(img_64, 1) / 255.0 - 1.0
    img_flat = np.transpose(img_flat, (2, 1, 0)).astype(np.float32)
    img_flat_rev = np.transpose(img_flat_rev, (2, 1, 0)).astype(np.float32)
        
    return img_flat, img_flat_rev

class CVBatch:
    def __init__(self, num):
        self.num = num
        self.images = np.zeros([num, INPUT_C, INPUT_W, INPUT_H]).astype(np.float32)
        self.nowCnt = 0
        self.nowIndex = 0
        self.gpu = False
        
        self.perm = np.random.permutation(self.num)
        
    def loadImage(self, fpass):
        
        self.images[self.nowCnt], self.images[self.nowCnt+1] = img_flat_read(fpass)
        self.nowCnt+=2
    
    def shuffle(self):
        np.random.shuffle(self.images)
    
    def nextBatch(self, n):

        if self.nowIndex+n >= self.nowCnt:
            
            self.perm = np.random.permutation(self.num)
            self.nowIndex = 0
            print("train_shuffle...")
        
        res = self.images[self.perm[self.nowIndex:self.nowIndex+n]]
        self.nowIndex += n
        #np.random.shuffle(self.images)
        return res
    
    
    
    def to_gpu(self):
        self.gpu = True
        self.images = xp.array(self.images, dtype=np.float32)
    
# ファイル全取得
def find_all_files(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            yield os.path.join(root, file)
    
fileList = find_all_files('train/')

i = 0
for file in sorted(fileList):
    if file.find('.png') != -1 or file.find('.jpg') != -1:
        i+=1
# 学習データ読み込み
total_num = i * 2
cvBatch = CVBatch(total_num)


print("TRAIN_TOTAL:" + str(total_num))


for file in find_all_files('train/'):
    
    if file.find('.png') != -1 or file.find('.jpg') != -1:
        print(file)
        #train_x[i],img = img_flat_read('train/' + file)
        CVBatch.loadImage(cvBatch, file)
        i+=1
#print("TRAIN_TOTAL:" + str(total_num))    
#CVBatch.to_gpu(cvBatch)




TRAIN_TOTAL:25004
train/cat.0.jpg
train/cat.1.jpg
train/cat.10.jpg
train/cat.100.jpg
train/cat.1000.jpg
train/cat.10000.jpg
train/cat.10001.jpg
train/cat.10002.jpg
train/cat.10003.jpg
train/cat.10004.jpg
train/cat.10005.jpg
train/cat.10006.jpg
train/cat.10007.jpg
train/cat.10008.jpg
train/cat.10009.jpg
train/cat.1001.jpg
train/cat.10010.jpg
train/cat.10011.jpg
train/cat.10012.jpg
train/cat.10013.jpg
train/cat.10014.jpg
train/cat.10015.jpg
train/cat.10016.jpg
train/cat.10017.jpg
train/cat.10018.jpg
train/cat.10019.jpg
train/cat.1002.jpg
train/cat.10020.jpg
train/cat.10021.jpg
train/cat.10022.jpg
train/cat.10023.jpg
train/cat.10024.jpg
train/cat.10025.jpg
train/cat.10026.jpg
train/cat.10027.jpg
train/cat.10028.jpg
train/cat.10029.jpg
train/cat.1003.jpg
train/cat.10030.jpg
train/cat.10031.jpg
train/cat.10032.jpg
train/cat.10033.jpg
train/cat.10034.jpg
train/cat.10035.jpg
train/cat.10036.jpg
train/cat.10037.jpg
train/cat.10038.jpg
train/cat.10039.jpg
train/cat.1004.jpg
train/cat.10040.jpg


train/cat.10370.jpg
train/cat.10371.jpg
train/cat.10372.jpg
train/cat.10373.jpg
train/cat.10374.jpg
train/cat.10375.jpg
train/cat.10376.jpg
train/cat.10377.jpg
train/cat.10378.jpg
train/cat.10379.jpg
train/cat.1038.jpg
train/cat.10380.jpg
train/cat.10381.jpg
train/cat.10382.jpg
train/cat.10383.jpg
train/cat.10384.jpg
train/cat.10385.jpg
train/cat.10386.jpg
train/cat.10387.jpg
train/cat.10388.jpg
train/cat.10389.jpg
train/cat.1039.jpg
train/cat.10390.jpg
train/cat.10391.jpg
train/cat.10392.jpg
train/cat.10393.jpg
train/cat.10394.jpg
train/cat.10395.jpg
train/cat.10396.jpg
train/cat.10397.jpg
train/cat.10398.jpg
train/cat.10399.jpg
train/cat.104.jpg
train/cat.1040.jpg
train/cat.10400.jpg
train/cat.10401.jpg
train/cat.10402.jpg
train/cat.10403.jpg
train/cat.10404.jpg
train/cat.10405.jpg
train/cat.10406.jpg
train/cat.10407.jpg
train/cat.10408.jpg
train/cat.10409.jpg
train/cat.1041.jpg
train/cat.10410.jpg
train/cat.10411.jpg
train/cat.10412.jpg
train/cat.10413.jpg
train/cat.10414.jpg
train/

train/cat.10745.jpg
train/cat.10746.jpg
train/cat.10747.jpg
train/cat.10748.jpg
train/cat.10749.jpg
train/cat.1075.jpg
train/cat.10750.jpg
train/cat.10751.jpg
train/cat.10752.jpg
train/cat.10753.jpg
train/cat.10754.jpg
train/cat.10755.jpg
train/cat.10756.jpg
train/cat.10757.jpg
train/cat.10758.jpg
train/cat.10759.jpg
train/cat.1076.jpg
train/cat.10760.jpg
train/cat.10761.jpg
train/cat.10762.jpg
train/cat.10763.jpg
train/cat.10764.jpg
train/cat.10765.jpg
train/cat.10766.jpg
train/cat.10767.jpg
train/cat.10768.jpg
train/cat.10769.jpg
train/cat.1077.jpg
train/cat.10770.jpg
train/cat.10771.jpg
train/cat.10772.jpg
train/cat.10773.jpg
train/cat.10774.jpg
train/cat.10775.jpg
train/cat.10776.jpg
train/cat.10777.jpg
train/cat.10778.jpg
train/cat.10779.jpg
train/cat.1078.jpg
train/cat.10780.jpg
train/cat.10781.jpg
train/cat.10782.jpg
train/cat.10783.jpg
train/cat.10784.jpg
train/cat.10785.jpg
train/cat.10786.jpg
train/cat.10787.jpg
train/cat.10788.jpg
train/cat.10789.jpg
train/cat.1079.jpg
train

train/cat.1112.jpg
train/cat.11120.jpg
train/cat.11121.jpg
train/cat.11122.jpg
train/cat.11123.jpg
train/cat.11124.jpg
train/cat.11125.jpg
train/cat.11126.jpg
train/cat.11127.jpg
train/cat.11128.jpg
train/cat.11129.jpg
train/cat.1113.jpg
train/cat.11130.jpg
train/cat.11131.jpg
train/cat.11132.jpg
train/cat.11133.jpg
train/cat.11134.jpg
train/cat.11135.jpg
train/cat.11136.jpg
train/cat.11137.jpg
train/cat.11138.jpg
train/cat.11139.jpg
train/cat.1114.jpg
train/cat.11140.jpg
train/cat.11141.jpg
train/cat.11142.jpg
train/cat.11143.jpg
train/cat.11144.jpg
train/cat.11145.jpg
train/cat.11146.jpg
train/cat.11147.jpg
train/cat.11148.jpg
train/cat.11149.jpg
train/cat.1115.jpg
train/cat.11150.jpg
train/cat.11151.jpg
train/cat.11152.jpg
train/cat.11153.jpg
train/cat.11154.jpg
train/cat.11155.jpg
train/cat.11156.jpg
train/cat.11157.jpg
train/cat.11158.jpg
train/cat.11159.jpg
train/cat.1116.jpg
train/cat.11160.jpg
train/cat.11161.jpg
train/cat.11162.jpg
train/cat.11163.jpg
train/cat.11164.jpg
train

train/cat.11495.jpg
train/cat.11496.jpg
train/cat.11497.jpg
train/cat.11498.jpg
train/cat.11499.jpg
train/cat.115.jpg
train/cat.1150.jpg
train/cat.11500.jpg
train/cat.11501.jpg
train/cat.11502.jpg
train/cat.11503.jpg
train/cat.11504.jpg
train/cat.11505.jpg
train/cat.11506.jpg
train/cat.11507.jpg
train/cat.11508.jpg
train/cat.11509.jpg
train/cat.1151.jpg
train/cat.11510.jpg
train/cat.11511.jpg
train/cat.11512.jpg
train/cat.11513.jpg
train/cat.11514.jpg
train/cat.11515.jpg
train/cat.11516.jpg
train/cat.11517.jpg
train/cat.11518.jpg
train/cat.11519.jpg
train/cat.1152.jpg
train/cat.11520.jpg
train/cat.11521.jpg
train/cat.11522.jpg
train/cat.11523.jpg
train/cat.11524.jpg
train/cat.11525.jpg
train/cat.11526.jpg
train/cat.11527.jpg
train/cat.11528.jpg
train/cat.11529.jpg
train/cat.1153.jpg
train/cat.11530.jpg
train/cat.11531.jpg
train/cat.11532.jpg
train/cat.11533.jpg
train/cat.11534.jpg
train/cat.11535.jpg
train/cat.11536.jpg
train/cat.11537.jpg
train/cat.11538.jpg
train/cat.11539.jpg
train/

...

train/cat.8129.jpg
train/cat.813.jpg
train/cat.8130.jpg
train/cat.8131.jpg
train/cat.8132.jpg
train/cat.8133.jpg
train/cat.8134.jpg
train/cat.8135.jpg
train/cat.8136.jpg
train/cat.8137.jpg
train/cat.8138.jpg
train/cat.8139.jpg
train/cat.814.jpg
train/cat.8140.jpg
train/cat.8141.jpg
train/cat.8142.jpg
train/cat.8143.jpg
train/cat.8144.jpg
train/cat.8145.jpg
train/cat.8146.jpg
train/cat.8147.jpg
train/cat.8148.jpg
train/cat.8149.jpg
train/cat.815.jpg
train/cat.8150.jpg
train/cat.8151.jpg
train/cat.8152.jpg
train/cat.8153.jpg
train/cat.8154.jpg
train/cat.8155.jpg
train/cat.8156.jpg
train/cat.8157.jpg
train/cat.8158.jpg
train/cat.8159.jpg
train/cat.816.jpg
train/cat.8160.jpg
train/cat.8161.jpg
train/cat.8162.jpg
train/cat.8163.jpg
train/cat.8164.jpg
train/cat.8165.jpg
train/cat.8166.jpg
train/cat.8167.jpg
train/cat.8168.jpg
train/cat.8169.jpg
train/cat.817.jpg
train/cat.8170.jpg
train/cat.8171.jpg
train/cat.8172.jpg
train/cat.8173.jpg
train/cat.8174.jpg
train/cat.8175.jpg
train/cat.8176.jp

train/cat.8525.jpg
train/cat.8526.jpg
train/cat.8527.jpg
train/cat.8528.jpg
train/cat.8529.jpg
train/cat.853.jpg
train/cat.8530.jpg
train/cat.8531.jpg
train/cat.8532.jpg
train/cat.8533.jpg
train/cat.8534.jpg
train/cat.8535.jpg
train/cat.8536.jpg
train/cat.8537.jpg
train/cat.8538.jpg
train/cat.8539.jpg
train/cat.854.jpg
train/cat.8540.jpg
train/cat.8541.jpg
train/cat.8542.jpg
train/cat.8543.jpg
train/cat.8544.jpg
train/cat.8545.jpg
train/cat.8546.jpg
train/cat.8547.jpg
train/cat.8548.jpg
train/cat.8549.jpg
train/cat.855.jpg
train/cat.8550.jpg
train/cat.8551.jpg
train/cat.8552.jpg
train/cat.8553.jpg
train/cat.8554.jpg
train/cat.8555.jpg
train/cat.8556.jpg
train/cat.8557.jpg
train/cat.8558.jpg
train/cat.8559.jpg
train/cat.856.jpg
train/cat.8560.jpg
train/cat.8561.jpg
train/cat.8562.jpg
train/cat.8563.jpg
train/cat.8564.jpg
train/cat.8565.jpg
train/cat.8566.jpg
train/cat.8567.jpg
train/cat.8568.jpg
train/cat.8569.jpg
train/cat.857.jpg
train/cat.8570.jpg
train/cat.8571.jpg
train/cat.8572.jp

train/cat.8921.jpg
train/cat.8922.jpg
train/cat.8923.jpg
train/cat.8924.jpg
train/cat.8925.jpg
train/cat.8926.jpg
train/cat.8927.jpg
train/cat.8928.jpg
train/cat.8929.jpg
train/cat.893.jpg
train/cat.8930.jpg
train/cat.8931.jpg
train/cat.8932.jpg
train/cat.8933.jpg
train/cat.8934.jpg
train/cat.8935.jpg
train/cat.8936.jpg
train/cat.8937.jpg
train/cat.8938.jpg
train/cat.8939.jpg
train/cat.894.jpg
train/cat.8940.jpg
train/cat.8941.jpg
train/cat.8942.jpg
train/cat.8943.jpg
train/cat.8944.jpg
train/cat.8945.jpg
train/cat.8946.jpg
train/cat.8947.jpg
train/cat.8948.jpg
train/cat.8949.jpg
train/cat.895.jpg
train/cat.8950.jpg
train/cat.8951.jpg
train/cat.8952.jpg
train/cat.8953.jpg
train/cat.8954.jpg
train/cat.8955.jpg
train/cat.8956.jpg
train/cat.8957.jpg
train/cat.8958.jpg
train/cat.8959.jpg
train/cat.896.jpg
train/cat.8960.jpg
train/cat.8961.jpg
train/cat.8962.jpg
train/cat.8963.jpg
train/cat.8964.jpg
train/cat.8965.jpg
train/cat.8966.jpg
train/cat.8967.jpg
train/cat.8968.jpg
train/cat.8969.j

train/cat.9317.jpg
train/cat.9318.jpg
train/cat.9319.jpg
train/cat.932.jpg
train/cat.9320.jpg
train/cat.9321.jpg
train/cat.9322.jpg
train/cat.9323.jpg
train/cat.9324.jpg
train/cat.9325.jpg
train/cat.9326.jpg
train/cat.9327.jpg
train/cat.9328.jpg
train/cat.9329.jpg
train/cat.933.jpg
train/cat.9330.jpg
train/cat.9331.jpg
train/cat.9332.jpg
train/cat.9333.jpg
train/cat.9334.jpg
train/cat.9335.jpg
train/cat.9336.jpg
train/cat.9337.jpg
train/cat.9338.jpg
train/cat.9339.jpg
train/cat.934.jpg
train/cat.9340.jpg
train/cat.9341.jpg
train/cat.9342.jpg
train/cat.9343.jpg
train/cat.9344.jpg
train/cat.9345.jpg
train/cat.9346.jpg
train/cat.9347.jpg
train/cat.9348.jpg
train/cat.9349.jpg
train/cat.935.jpg
train/cat.9350.jpg
train/cat.9351.jpg
train/cat.9352.jpg
train/cat.9353.jpg
train/cat.9354.jpg
train/cat.9355.jpg
train/cat.9356.jpg
train/cat.9357.jpg
train/cat.9358.jpg
train/cat.9359.jpg
train/cat.936.jpg
train/cat.9360.jpg
train/cat.9361.jpg
train/cat.9362.jpg
train/cat.9363.jpg
train/cat.9364.jp

train/cat.9709.jpg
train/cat.971.jpg
train/cat.9710.jpg
train/cat.9711.jpg
train/cat.9712.jpg
train/cat.9713.jpg
train/cat.9714.jpg
train/cat.9715.jpg
train/cat.9716.jpg
train/cat.9717.jpg
train/cat.9718.jpg
train/cat.9719.jpg
train/cat.972.jpg
train/cat.9720.jpg
train/cat.9721.jpg
train/cat.9722.jpg
train/cat.9723.jpg
train/cat.9724.jpg
train/cat.9725.jpg
train/cat.9726.jpg
train/cat.9727.jpg
train/cat.9728.jpg
train/cat.9729.jpg
train/cat.973.jpg
train/cat.9730.jpg
train/cat.9731.jpg
train/cat.9732.jpg
train/cat.9733.jpg
train/cat.9734.jpg
train/cat.9735.jpg
train/cat.9736.jpg
train/cat.9737.jpg
train/cat.9738.jpg
train/cat.9739.jpg
train/cat.974.jpg
train/cat.9740.jpg
train/cat.9741.jpg
train/cat.9742.jpg
train/cat.9743.jpg
train/cat.9744.jpg
train/cat.9745.jpg
train/cat.9746.jpg
train/cat.9747.jpg
train/cat.9748.jpg
train/cat.9749.jpg
train/cat.975.jpg
train/cat.9750.jpg
train/cat.9751.jpg
train/cat.9752.jpg
train/cat.9753.jpg
train/cat.9754.jpg
train/cat.9755.jpg
train/cat.9756.jp

In [13]:
def real_image(images, stage):


    if math.floor(stage)%2==0:
        reso = min(256, 4 * 2**(((math.floor(stage)+1)//2)))
        scale = max(1, 256//reso)
        if scale>1:
            images = F.average_pooling_2d(images, scale, scale, 0)
    else:
        alpha = stage - math.floor(stage)
        reso_low = min(256, 4 * 2**(((math.floor(stage))//2)))
        reso_high = min(256, 4 * 2**(((math.floor(stage)+1)//2)))
        scale_low = max(1, 256//reso_low)
        scale_high = max(1, 256//reso_high)
        
        images_low = F.unpooling_2d(F.average_pooling_2d(x_real, scale_low, scale_low, 0), 2, 2, 0, outsize=(reso_high, reso_high))
        
        if scale_high > 1:
            images_high = F.average_pooling_2d(images, scale_high, scale_high, 0)
        else:
            images_high = images
        images = (1-alpha)*images_low + alpha*images_high
        
    return images

In [14]:
print(total_num)

25004


In [15]:
# シード値設定
np.random.seed(seed=20180101)
xp.random.seed(seed=20180101)
print(sum(p.data.size for p in gen.params()))
print(sum(p.data.size for p in cri.params()))

23072120
23080225


In [16]:
def getCount(init, batches, interval):
    counter = 0
    hoge = init
    for i in range(len(batches)*2):
        tmp = interval / batches[i//2]

        hoge = hoge - tmp
        if hoge > 0:
            counter += interval
        else:
            hoge = hoge + tmp
            counter += hoge*batches[i//2]
            break
    return (int)(counter)

In [17]:
batch_size = 16
batches = (16,16,16,16,16,16,9)
n_critic = 1

g_loss = 0
c_loss = 0
w_dis = 0
w_dis_sum = 0

grad_loss = 0

smoothing = 0.999

zTest = generator_smooth.make_hidden(16)
zTest = Variable(zTest)

stage=0
if os.path.isfile('stage.txt') and init!=0:
    stage = np.loadtxt('stage.txt')
    
test_interval = 100

save_interval = 5000

# w_dis保存用
w_dis_list = np.empty((0,2))
if os.path.isfile('log.csv') and init!=0:
    w_dis_list = np.loadtxt('log.csv', delimiter=',')


# 時間計測
A = time.time()

# stage学習枚数
interval = 400000
counter = getCount(init, batches, interval)
    
for i in range(10000000):
    
    stage = counter / interval
    
    if stage >= 13:
        stage = 12
    
    batch_size = batches[int((stage+1)//2)]
    
    # critic更新   
    #stage=3
    
    # 本物用
    x_real = cvBatch.nextBatch(batch_size)
    x_real = Variable(xp.asarray(x_real, dtype=np.float32))

    # スケール合わせ
    x_real = real_image(x_real, stage)
    y_real = cri(x_real, stage=stage)
    
    z = gen.make_hidden(batch_size)
    z = Variable(z)

    # 偽物用
    x_fake = gen(z, stage=stage)
    #print(x_real.data.shape, x_fake.data.shape)
    y_fake = cri(x_fake, stage=stage)

    # BackPropの参照を削除
    x_fake.unchain_backward()

    # grad
    eps = xp.random.uniform(0, 1, size=batch_size).astype("f")[:, None, None, None]
    x_mid = eps * x_real + (1.0 - eps) * x_fake

    x_mid_v = Variable(x_mid.data)
    y_mid = F.sum(cri(x_mid_v, stage=stage))

    dydx, = chainer.grad([y_mid], [x_mid_v], enable_double_backprop=True)
    dydx = F.sqrt(F.sum(dydx*dydx, axis=(1, 2, 3)))
    #loss_grad = 10.0 * F.mean_squared_error(dydx, 750.0 * xp.ones_like(dydx.data)) * (1.0/750.0**2)
    loss_grad = 10.0 * F.mean_squared_error(dydx, xp.ones_like(dydx.data))

    # 0.001
    loss_dr = 0.001 * F.sum(y_real**2) / batch_size

    L_cri = F.sum(-y_real) / batch_size
    L_cri += F.sum(y_fake) / batch_size

    L_cri += loss_dr

    c_loss += (L_cri.data)

    L_cri +=loss_grad

    grad_loss+=loss_grad.data
    
    # w距離
    w = F.sum(-y_fake).data / batch_size +  F.sum(y_real).data / batch_size
    w_dis += w
    w_dis_sum += w

    cri.cleargrads()
    L_cri.backward()
    cri_optimizer.update()

    # BackPropの参照を削除
    L_cri.unchain_backward()
    
    # 偽物用
    z = gen.make_hidden(batch_size)
    z = Variable(z)
        
    x_fake = gen(z, stage=stage)
    y_fake = cri(x_fake, stage=stage)

    L_gen = F.sum(-y_fake) / batch_size
    
    g_loss += (L_gen.data)
    
    gen.cleargrads()
    L_gen.backward()
    gen_optimizer.update()
    
    # BackPropの参照を削除
    L_gen.unchain_backward()
    
    # update smoothed generator
    soft_copy_param(generator_smooth, gen, 1.0-smoothing)
    
    if (i+init) % save_interval == 0 and i != 0:
        print("save")
        gen.to_cpu()
        serializers.save_npz("generator.npz", gen)
        gen.to_gpu(gpu)
        
        generator_smooth.to_cpu()
        serializers.save_npz("generator_smooth.npz", generator_smooth)
        generator_smooth.to_gpu(gpu)
        
        cri.to_cpu()
        serializers.save_npz("critic.npz", cri)
        cri.to_gpu(gpu)
        # 現在のステージを保存
        np.savetxt('stage.txt', np.array([stage]))
        print("ok")
        
    if i % 500 == 0 and i != 0:
        # w_dis保存
        w_dis_list = np.append(w_dis_list, np.array([[int(init+i), w_dis_sum/500]]), axis=0)
        np.savetxt('log.csv', w_dis_list,delimiter=',')
        w_dis_sum = 0
    
    if (init+i) % 10000 == 0 and i != 0:
        print("save")
        gen.to_cpu()
        serializers.save_npz("./weight/generator_" + str(init+i) + ".npz", gen)
        gen.to_gpu(gpu)
        
        generator_smooth.to_cpu()
        serializers.save_npz("./weight/generator_smooth_" + str(init+i) + ".npz", generator_smooth)
        generator_smooth.to_gpu(gpu)
        
        cri.to_cpu()
        serializers.save_npz("./weight/critic_" + str(init+i) + ".npz", cri)
        cri.to_gpu(gpu)
        # 現在のステージを保存
        np.savetxt('stage.txt', np.array([stage]))
        print("ok")
    
    if i % test_interval == 0:

        # 画像出力
        # テストモードに
        with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
            
            # 偽物用
            x_test = generator_smooth(zTest, stage)
            #y_test = cri(x_test)
            
            size = 4*2**((int(stage)+1)//2)
            
            print(str(init+i) + ",epoch:" + str(int(batch_size/total_num*(init+i))) + ",gen:" + str(g_loss/test_interval) + 
                  ", cri:" + str(c_loss/test_interval) + ", w_dis:" + str(w_dis/test_interval) + ", grad_loss:" + str(grad_loss/test_interval))
            print("stage:" + str(stage) + ", size:" + str(size))
            
            g_loss = 0
            c_loss = 0
            w_dis = 0
            grad_loss = 0
            
        imgs = cuda.to_cpu(x_test.data)
        imgs = np.transpose(imgs, (0, 3, 2, 1))

        # 画像を連結
        img2 = []
        
        s = 4

        for k in range(s):
            img2.append(imgs[k*s])

            for j in range(s):
                if j != 0:
                    img2[k] = cv2.hconcat([img2[k], imgs[k*s+j]])
            if k == 0:
                img3 = img2[0]
            else:
                img3 = cv2.vconcat([img3, img2[k]])

        img3 = np.asarray(np.clip(img3 * 127.5 + 127.5, 0.0, 255.0), dtype=np.uint8)
        cv2.imwrite('./res/'+str(init+i)+'.jpg', img3)
        
        # かかった時間
        print("time:{0} sec".format(time.time()-A))
        
        A = time.time()
        
    counter+=batch_size

0,epoch:0,gen:0.0026182415895164013, cri:0.004774737171828747, w_dis:-0.004771891050040722, grad_loss:0.011655940674245358
stage:0.0, size:4
time:2.5918846130371094 sec
100,epoch:0,gen:3.116748571395874, cri:-2.6800777912139893, w_dis:2.680558443069458, grad_loss:0.5326367020606995
stage:0.004, size:4
time:11.701091289520264 sec
200,epoch:0,gen:1.2801910638809204, cri:-1.6275759935379028, w_dis:1.628363847732544, grad_loss:0.3142055571079254
stage:0.008, size:4
time:11.669005870819092 sec
300,epoch:0,gen:0.34298938512802124, cri:-1.1047720909118652, w_dis:1.106137990951538, grad_loss:0.23080331087112427
stage:0.012, size:4
time:12.660345077514648 sec
400,epoch:0,gen:0.09022372961044312, cri:-0.8393170237541199, w_dis:0.8407076001167297, grad_loss:0.1849583387374878
stage:0.016, size:4
time:12.200273752212524 sec
500,epoch:0,gen:0.13137201964855194, cri:-0.5305470824241638, w_dis:0.5312334299087524, grad_loss:0.14898478984832764
stage:0.02, size:4
time:11.730167627334595 sec
600,epoch:0

4900,epoch:3,gen:0.2896687388420105, cri:-0.19374260306358337, w_dis:0.19391924142837524, grad_loss:0.08501642942428589
stage:0.196, size:4
time:11.74263882637024 sec
save
ok
5000,epoch:3,gen:0.2431354969739914, cri:-0.19924931228160858, w_dis:0.1994124799966812, grad_loss:0.0804915726184845
stage:0.2, size:4
time:29.441588163375854 sec
5100,epoch:3,gen:0.2218426764011383, cri:-0.15972355008125305, w_dis:0.15984830260276794, grad_loss:0.0755653828382492
stage:0.204, size:4
time:11.88190507888794 sec
5200,epoch:3,gen:0.31192007660865784, cri:-0.19831746816635132, w_dis:0.1984817385673523, grad_loss:0.0774448961019516
stage:0.208, size:4
time:11.798802137374878 sec
5300,epoch:3,gen:0.22297894954681396, cri:-0.1952798217535019, w_dis:0.19543220102787018, grad_loss:0.08352749794721603
stage:0.212, size:4
time:11.93679404258728 sec
5400,epoch:3,gen:0.2190943956375122, cri:-0.18987590074539185, w_dis:0.1900416761636734, grad_loss:0.08459636569023132
stage:0.216, size:4
time:11.83256888389587

9800,epoch:6,gen:0.20276089012622833, cri:-0.20863939821720123, w_dis:0.2087649554014206, grad_loss:0.07071468979120255
stage:0.392, size:4
time:11.870521545410156 sec
9900,epoch:6,gen:0.2837282717227936, cri:-0.20780932903289795, w_dis:0.2079542577266693, grad_loss:0.06896389275789261
stage:0.396, size:4
time:11.82843017578125 sec
save
ok
save
ok
10000,epoch:6,gen:0.26405513286590576, cri:-0.21699856221675873, w_dis:0.21714119613170624, grad_loss:0.07413638383150101
stage:0.4, size:4
time:46.29659914970398 sec
10100,epoch:6,gen:0.26334530115127563, cri:-0.20951201021671295, w_dis:0.20968374609947205, grad_loss:0.06955180317163467
stage:0.404, size:4
time:12.032516956329346 sec
10200,epoch:6,gen:0.21599890291690826, cri:-0.21767842769622803, w_dis:0.21783427894115448, grad_loss:0.0734671801328659
stage:0.408, size:4
time:11.79835057258606 sec
10300,epoch:6,gen:0.23316994309425354, cri:-0.2102586030960083, w_dis:0.2104048877954483, grad_loss:0.06745395809412003
stage:0.412, size:4
time:

14700,epoch:9,gen:0.28346362709999084, cri:-0.20251327753067017, w_dis:0.2026708573102951, grad_loss:0.06572198122739792
stage:0.588, size:4
time:11.516599893569946 sec
14800,epoch:9,gen:0.22380080819129944, cri:-0.23361323773860931, w_dis:0.23376651108264923, grad_loss:0.06644609570503235
stage:0.592, size:4
time:11.726158142089844 sec
14900,epoch:9,gen:0.2080242931842804, cri:-0.19767093658447266, w_dis:0.1977887898683548, grad_loss:0.0610392726957798
stage:0.596, size:4
time:11.694074153900146 sec
save
ok
15000,epoch:9,gen:0.20332589745521545, cri:-0.19874507188796997, w_dis:0.19885970652103424, grad_loss:0.060753822326660156
stage:0.6, size:4
time:28.7814781665802 sec
15100,epoch:9,gen:0.26379555463790894, cri:-0.20700420439243317, w_dis:0.20714665949344635, grad_loss:0.05852406099438667
stage:0.604, size:4
time:11.676025390625 sec
15200,epoch:9,gen:0.1881037950515747, cri:-0.20458705723285675, w_dis:0.20472906529903412, grad_loss:0.05719853565096855
stage:0.608, size:4
time:11.621

19600,epoch:12,gen:0.14497342705726624, cri:-0.17311054468154907, w_dis:0.17326240241527557, grad_loss:0.061332106590270996
stage:0.784, size:4
time:13.233118295669556 sec
19700,epoch:12,gen:0.21619576215744019, cri:-0.16844680905342102, w_dis:0.16857481002807617, grad_loss:0.06416821479797363
stage:0.788, size:4
time:12.883281946182251 sec
19800,epoch:12,gen:0.2141939401626587, cri:-0.20171646773815155, w_dis:0.20186133682727814, grad_loss:0.06769503653049469
stage:0.792, size:4
time:12.298206329345703 sec
19900,epoch:12,gen:0.22639566659927368, cri:-0.17478854954242706, w_dis:0.17493633925914764, grad_loss:0.07028449326753616
stage:0.796, size:4
time:11.961563110351562 sec
save
ok
save
ok
20000,epoch:12,gen:0.20423686504364014, cri:-0.15804576873779297, w_dis:0.1582086980342865, grad_loss:0.06379106640815735
stage:0.8, size:4
time:47.02813959121704 sec
20100,epoch:12,gen:0.10958074778318405, cri:-0.14624838531017303, w_dis:0.1463918834924698, grad_loss:0.06989360600709915
stage:0.804

...

139900,epoch:89,gen:0.36576953530311584, cri:-0.5176169872283936, w_dis:0.5269553065299988, grad_loss:0.062397945672273636
stage:5.596, size:32
time:64.12539291381836 sec
save
ok
save
ok
140000,epoch:89,gen:0.9055685997009277, cri:-0.5669071078300476, w_dis:0.5751880407333374, grad_loss:0.06515835970640182
stage:5.6, size:32
time:97.77179622650146 sec
140100,epoch:89,gen:0.34301143884658813, cri:-0.6711308360099792, w_dis:0.680690586566925, grad_loss:0.06747709959745407
stage:5.604, size:32
time:63.74337887763977 sec
140200,epoch:89,gen:0.9754431247711182, cri:-0.574027419090271, w_dis:0.5845444202423096, grad_loss:0.06799927353858948
stage:5.608, size:32
time:64.03916263580322 sec
140300,epoch:89,gen:0.6620466709136963, cri:-0.5171756148338318, w_dis:0.5266751646995544, grad_loss:0.06524041295051575
stage:5.612, size:32
time:64.10634279251099 sec
140400,epoch:89,gen:1.4452821016311646, cri:-0.6654235124588013, w_dis:0.6746433973312378, grad_loss:0.06835083663463593
stage:5.616, size:3

144800,epoch:92,gen:0.7634012699127197, cri:-0.5118445158004761, w_dis:0.5208386778831482, grad_loss:0.0676613450050354
stage:5.792, size:32
time:65.70925831794739 sec
144900,epoch:92,gen:0.99480140209198, cri:-0.5675686001777649, w_dis:0.5761643052101135, grad_loss:0.06555286049842834
stage:5.796, size:32
time:65.64221787452698 sec
save
ok
145000,epoch:92,gen:0.8606257438659668, cri:-0.6362916827201843, w_dis:0.6441792845726013, grad_loss:0.0629558116197586
stage:5.8, size:32
time:80.47319269180298 sec
145100,epoch:92,gen:0.4104793667793274, cri:-0.5529297590255737, w_dis:0.5622993111610413, grad_loss:0.06677969545125961
stage:5.804, size:32
time:65.40973854064941 sec
145200,epoch:92,gen:1.2208762168884277, cri:-0.555413544178009, w_dis:0.5645172595977783, grad_loss:0.06220234930515289
stage:5.808, size:32
time:65.67861342430115 sec
train_shuffle...
145300,epoch:92,gen:0.7149698138237, cri:-0.6161395311355591, w_dis:0.6260868906974792, grad_loss:0.06953470408916473
stage:5.812, size:3

149700,epoch:95,gen:0.487321674823761, cri:-0.6265981793403625, w_dis:0.6359988451004028, grad_loss:0.06930384039878845
stage:5.988, size:32
time:65.87860178947449 sec
149800,epoch:95,gen:1.0629349946975708, cri:-0.639958381652832, w_dis:0.6482923626899719, grad_loss:0.07116337865591049
stage:5.992, size:32
time:65.9065752029419 sec
149900,epoch:95,gen:0.7188625931739807, cri:-0.5892801880836487, w_dis:0.5989762544631958, grad_loss:0.07498467713594437
stage:5.996, size:32
time:65.94059371948242 sec
train_shuffle...
save
ok
save
ok
150000,epoch:95,gen:0.6506786346435547, cri:-0.6804115176200867, w_dis:0.690002977848053, grad_loss:0.06688854098320007
stage:6.0, size:32
time:95.14351892471313 sec
150100,epoch:96,gen:1.3594567775726318, cri:-0.6344367861747742, w_dis:0.6428464651107788, grad_loss:0.07141602784395218
stage:6.004, size:32
time:64.19001054763794 sec
150200,epoch:96,gen:0.8787046074867249, cri:-0.6137984395027161, w_dis:0.6234143972396851, grad_loss:0.06868371367454529
stage:6

154600,epoch:98,gen:1.0182324647903442, cri:-0.6343719363212585, w_dis:0.6427375674247742, grad_loss:0.06596826761960983
stage:6.184, size:32
time:64.61539483070374 sec
train_shuffle...
154700,epoch:98,gen:0.5493161082267761, cri:-0.6933031678199768, w_dis:0.7025254964828491, grad_loss:0.06575754284858704
stage:6.188, size:32
time:64.641193151474 sec
154800,epoch:99,gen:0.5781130790710449, cri:-0.7252440452575684, w_dis:0.7359199523925781, grad_loss:0.08741842210292816
stage:6.192, size:32
time:64.65967559814453 sec
154900,epoch:99,gen:0.8016956448554993, cri:-0.7083355784416199, w_dis:0.7189111113548279, grad_loss:0.06767065823078156
stage:6.196, size:32
time:64.64649653434753 sec
save
ok
155000,epoch:99,gen:1.0826412439346313, cri:-0.7400362491607666, w_dis:0.750446617603302, grad_loss:0.07260536402463913
stage:6.2, size:32
time:80.85967898368835 sec
155100,epoch:99,gen:0.9227604866027832, cri:-0.6092160940170288, w_dis:0.6188278198242188, grad_loss:0.06618829071521759
stage:6.204, s

KeyboardInterrupt: 

In [ ]:
gen.to_cpu()
serializers.save_npz("generator.npz", gen)
cri.to_cpu()
serializers.save_npz("critic.npz", cri)